In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn import preprocessing
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3  import InceptionV3
#from tensorflow.keras.applications.resnet50  import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
csv_data=pd.read_csv("/content/gdrive/My Drive/Dance_Form_Identification/train.csv")
csv_data.head()
lbl = preprocessing.LabelEncoder()
lbl.fit(list(csv_data['target'].values))
csv_data['target'] = lbl.transform(list(csv_data['target'].values))

In [ ]:
count=0
train_data=[]
y_train=[]
filename="/content/gdrive/My Drive/Dance_Form_Identification/train/"

for i in os.listdir(filename):
  try:
    count=count+1
    if count%100==0:
      print(f"{count} Images read")
    image = cv2.imread(filename+i)
    image_array = Image.fromarray(image)    
    resize_img = image_array.resize((256 , 256))
    train_data.append(resize_img)
    for j in range(len(csv_data)):
      if csv_data.iloc[j]['Image']==i:
        y_train.append(csv_data.iloc[j]['target'])
        break
    print(filename+i)
  except AttributeError:
    print("Exception")

y_train=np.asarray(y_train)

In [ ]:
from keras import backend as K
import keras
y_train=keras.utils.to_categorical(y_train,8)

In [ ]:
model=Sequential()
model.add(VGG16(include_top=False, input_shape=(256, 256, 3)))
# add new classifier layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(768, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(8, activation='softmax'))
# define new model

# summarize
model.summary()
model.layers[0].trainable = False

In [ ]:
final_data=[]
for i in range(len(train_data)):
  final_data.append(np.asarray(train_data[i]))

train_data=np.asarray(final_data)
train_data=train_data/255
train_data=train_data.reshape(-1, 256,256,3)*2. -1.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
test=pd.read_csv("/content/gdrive/My Drive/Dance_Form_Identification/test.csv")

file_path="/content/gdrive/My Drive/Dance_Form_Identification/test/"
test_data=[]
for i in range(len(test)):
  image = cv2.imread(file_path+test.iloc[i]['Image'])
  print(file_path+test.iloc[i]['Image'])
  image_array = Image.fromarray(image)    
  resize_img = image_array.resize((256 , 256))
  test_data.append(resize_img)

In [ ]:
final_data=[]
for i in range(len(test_data)):
  final_data.append(np.asarray(test_data[i]))

test_data=np.asarray(final_data)
test_data=test_data/255
test_data=test_data.reshape(-1, 256,256,3)*2. -1.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
train_datagenerator = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        rotation_range=40,  
        zoom_range = 0.20,  
        width_shift_range=0.05,  
        height_shift_range=0.075,  
        horizontal_flip=True,  
        vertical_flip=False) 

train_datagenerator.fit(train_data)

train_data_final=np.array(train_data)
print(train_data_final.shape)

In [ ]:
history =model.fit_generator(
    train_datagenerator.flow(train_data_final, y_train, batch_size=32), epochs=60)

In [ ]:
y_pred=model.predict(test_data)
y_pred_classes=[]
for i in range(len(y_pred)):
  y_pred_classes.append(np.argmax(y_pred[i]))

y_pred_classes=lbl.inverse_transform(y_pred_classes)
test['target']=y_pred_classes
test.to_csv("Submission.csv", index=False)